In [2]:
# Creating a table where all incidents are associated to a time

import sqlite3 as db 
import os
import pandas as pd

# Connect to database using absolute path
db_path = '../sql_databases/processed_data.db'
conn = db.connect(db_path)
cursor = conn.cursor()

# Execute query to combine all incidents with their timestamps
query = """
    SELECT 
        '311_service_requests' AS source_table,
        created_date AS incident_time,
        category AS incident_type,
        complaint_type AS description,
        incident_address AS address,
        neighborhood,
        latitude,
        longitude
    FROM "311_service_requests"
    WHERE created_date IS NOT NULL
    
    UNION ALL
    
    SELECT
        'fire_incidents' AS source_table,
        "Incident Date" AS incident_time,
        "Primary Situation" AS incident_type,
        "Action Taken Primary" AS description,
        Address AS address,
        "Analysis Neighborhood" AS neighborhood,
        NULL AS latitude,
        NULL AS longitude
    FROM fire_incidents
    WHERE "Incident Date" IS NOT NULL
    
    UNION ALL
    
    SELECT
        'fire_safety_complaints' AS source_table,
        "Received Date" AS incident_time,
        "Complaint Item Type Description" AS incident_type,
        Disposition AS description,
        Address AS address,
        "Neighborhood  District" AS neighborhood,
        NULL AS latitude,
        NULL AS longitude
    FROM fire_safety_complaints
    WHERE "Received Date" IS NOT NULL
    
    UNION ALL
    
    SELECT
        'fire_violations' AS source_table,
        "violation date" AS incident_time,
        "violation item description" AS incident_type,
        Status AS description,
        Address AS address,
        "neighborhood district" AS neighborhood,
        NULL AS latitude,
        NULL AS longitude
    FROM fire_violations
    WHERE "violation date" IS NOT NULL
    
    UNION ALL
    
    SELECT
        'sffd_service_calls' AS source_table,
        call_date AS incident_time,
        call_type AS incident_type,
        call_final_disposition AS description,
        address,
        supervisor_district AS neighborhood,
        latitude,
        longitude
    FROM sffd_service_calls
    WHERE call_date IS NOT NULL
    
    UNION ALL
    
    SELECT
        'sfpd_incidents' AS source_table,
        timestamp AS incident_time,
        category AS incident_type,
        descript AS description,
        address,
        pddistrict AS neighborhood,
        latitude,
        longitude
    FROM sfpd_incidents
    WHERE timestamp IS NOT NULL
    
    ORDER BY incident_time DESC
"""

# Execute and fetch results
df = pd.read_sql_query(query, conn)
conn.close()

# Display results
print(f"Total incidents: {len(df)}")
print(f"\nIncidents by source:")
print(df['source_table'].value_counts())
print(f"\nFirst 10 incidents:")
print(df.head(10))

# Save results to a new database
results_db_path = '/Users/amanarham/Documents/University_of_Pennsylvania/CIT-5500/Project/UPenn_500_Proj/sql_databases/results.db'

# Create connection to results database
results_conn = db.connect(results_db_path)

# Save DataFrame to the database as a table
df.to_sql('incidents_by_time', results_conn, if_exists='replace', index=False)

results_conn.close()

print(f"\n✓ Results saved to {results_db_path}")
print(f"  Table name: 'incidents_by_time'")
print(f"  Total rows: {len(df)}")

Total incidents: 256784

Incidents by source:
source_table
fire_incidents            148786
fire_safety_complaints     47204
fire_violations            30794
311_service_requests       10000
sfpd_incidents             10000
sffd_service_calls         10000
Name: count, dtype: int64

First 10 incidents:
             source_table            incident_time  \
0          fire_incidents  2020-09-11T00:00:00.000   
1          fire_incidents  2020-09-11T00:00:00.000   
2  fire_safety_complaints  2020-09-11T00:00:00.000   
3  fire_safety_complaints  2020-09-11T00:00:00.000   
4  fire_safety_complaints  2020-09-11T00:00:00.000   
5          fire_incidents  2020-09-10T00:00:00.000   
6          fire_incidents  2020-09-10T00:00:00.000   
7          fire_incidents  2020-09-10T00:00:00.000   
8          fire_incidents  2020-09-10T00:00:00.000   
9          fire_incidents  2020-09-10T00:00:00.000   

                                       incident_type  \
0                                    100 Fire

In [3]:
# Create a table generating a list for the top areas where the most incidents have occurred

import sqlite3 as db 
import pandas as pd

# Connect to results database
results_db_path = '/Users/amanarham/Documents/University_of_Pennsylvania/CIT-5500/Project/UPenn_500_Proj/sql_databases/results.db'
conn = db.connect(results_db_path)

# Query to get top areas by incident count
query = """
    SELECT 
        neighborhood,
        COUNT(*) as incident_count,
        COUNT(DISTINCT source_table) as data_sources,
        COUNT(DISTINCT incident_type) as incident_types
    FROM incidents_by_time
    WHERE neighborhood IS NOT NULL AND neighborhood != ''
    GROUP BY neighborhood
    ORDER BY incident_count DESC
"""

# Execute query
df_top_areas = pd.read_sql_query(query, conn)

# Display top 20 areas
print("Top 20 Areas by Incident Count:")
print("="*60)
print(df_top_areas.head(20).to_string(index=False))

# Save to results database
df_top_areas.to_sql('top_areas_by_incidents', conn, if_exists='replace', index=False)

conn.close()

print(f"\n✓ Results saved to table 'top_areas_by_incidents'")
print(f"  Total areas: {len(df_top_areas)}")

# Display summary statistics
print(f"\nSummary Statistics:")
print(f"  Average incidents per area: {df_top_areas['incident_count'].mean():.2f}")
print(f"  Median incidents per area: {df_top_areas['incident_count'].median():.2f}")
print(f"  Max incidents in single area: {df_top_areas['incident_count'].max()}")
print(f"  Min incidents in single area: {df_top_areas['incident_count'].min()}")

df_top_areas.head(20)

Top 20 Areas by Incident Count:
                  neighborhood  incident_count  data_sources  incident_types
Financial District/South Beach           23654             3             376
                    Tenderloin           21144             4             377
                       Mission           19106             4             378
               South of Market           15905             4             348
         Bayview Hunters Point           11290             3             289
                          PARK           10000             1              35
                      Nob Hill            9985             4             303
              Western Addition            8448             4             278
               Pacific Heights            7885             4             308
               Sunset/Parkside            7480             3             237
                     Chinatown            7391             4             319
                        Marina            68

,neighborhood,incident_count,data_sources,incident_types
0,Financial District/South Beach,23654,3,376
1,Tenderloin,21144,4,377
2,Mission,19106,4,378
3,South of Market,15905,4,348
4,Bayview Hunters Point,11290,3,289
5,PARK,10000,1,35
6,Nob Hill,9985,4,303
7,Western Addition,8448,4,278
8,Pacific Heights,7885,4,308
9,Sunset/Parkside,7480,3,237


In [4]:
# Create a table showing which neighborhoods are most dangerous at different times

import sqlite3 as db 
import pandas as pd

# Connect to results database
results_db_path = '/Users/amanarham/Documents/University_of_Pennsylvania/CIT-5500/Project/UPenn_500_Proj/sql_databases/results.db'
conn = db.connect(results_db_path)

# Query to analyze incidents by time of day and day of week
query = """
    WITH time_parsed AS (
        SELECT 
            neighborhood,
            incident_time,
            incident_type,
            CASE 
                WHEN CAST(SUBSTR(incident_time, 12, 2) AS INTEGER) BETWEEN 6 AND 11 THEN 'Morning (6am-11am)'
                WHEN CAST(SUBSTR(incident_time, 12, 2) AS INTEGER) BETWEEN 12 AND 17 THEN 'Afternoon (12pm-5pm)'
                WHEN CAST(SUBSTR(incident_time, 12, 2) AS INTEGER) BETWEEN 18 AND 21 THEN 'Evening (6pm-9pm)'
                ELSE 'Night (10pm-5am)'
            END AS time_period,
            CASE 
                WHEN CAST(strftime('%w', incident_time) AS INTEGER) IN (0, 6) THEN 'Weekend'
                ELSE 'Weekday'
            END AS day_type
        FROM incidents_by_time
        WHERE neighborhood IS NOT NULL 
            AND neighborhood != ''
            AND incident_time IS NOT NULL
            AND incident_time != ''
    )
    SELECT 
        neighborhood,
        time_period,
        day_type,
        COUNT(*) as incident_count,
        COUNT(DISTINCT incident_type) as incident_types,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY neighborhood), 2) as pct_of_neighborhood_incidents
    FROM time_parsed
    GROUP BY neighborhood, time_period, day_type
    ORDER BY neighborhood, incident_count DESC
"""

# Execute query
df_time_danger = pd.read_sql_query(query, conn)

# Display overall statistics
print("Time-Based Danger Analysis")
print("="*80)
print(f"\nTotal records analyzed: {len(df_time_danger)}")

# Show top 10 most dangerous neighborhood-time combinations
print("\nTop 10 Most Dangerous Neighborhood-Time Combinations:")
print("-"*80)
top_10 = df_time_danger.nlargest(10, 'incident_count')
print(top_10.to_string(index=False))

# Analysis by time period
print("\n\nIncidents by Time Period (All Neighborhoods):")
print("-"*80)
time_summary = df_time_danger.groupby('time_period')['incident_count'].sum().sort_values(ascending=False)
print(time_summary)

# Analysis by day type
print("\n\nIncidents by Day Type (All Neighborhoods):")
print("-"*80)
day_summary = df_time_danger.groupby('day_type')['incident_count'].sum().sort_values(ascending=False)
print(day_summary)

# Find neighborhoods with significant night danger
print("\n\nNeighborhoods Most Dangerous at Night:")
print("-"*80)
night_danger = df_time_danger[df_time_danger['time_period'] == 'Night (10pm-5am)'].nlargest(10, 'incident_count')
print(night_danger[['neighborhood', 'day_type', 'incident_count', 'pct_of_neighborhood_incidents']].to_string(index=False))

# Save to results database
df_time_danger.to_sql('time_based_danger_zones', conn, if_exists='replace', index=False)

conn.close()

print(f"\n\n✓ Results saved to table 'time_based_danger_zones'")
print(f"  Total neighborhood-time combinations: {len(df_time_danger)}")

df_time_danger.head(20)

Time-Based Danger Analysis

Total records analyzed: 790

Top 10 Most Dangerous Neighborhood-Time Combinations:
--------------------------------------------------------------------------------
                  neighborhood      time_period day_type  incident_count  incident_types  pct_of_neighborhood_incidents
Financial District/South Beach Night (10pm-5am)  Weekday           20385             365                          86.18
                    Tenderloin Night (10pm-5am)  Weekday           16873             366                          79.80
                       Mission Night (10pm-5am)  Weekday           15209             362                          79.60
               South of Market Night (10pm-5am)  Weekday           12539             338                          78.84
         Bayview Hunters Point Night (10pm-5am)  Weekday            8872             280                          78.58
                      Nob Hill Night (10pm-5am)  Weekday            8271             293

,neighborhood,time_period,day_type,incident_count,incident_types,pct_of_neighborhood_incidents
0,1,Night (10pm-5am),Weekday,222,7,80.73
1,1,Night (10pm-5am),Weekend,53,5,19.27
2,10,Night (10pm-5am),Weekday,788,18,81.07
3,10,Night (10pm-5am),Weekend,184,11,18.93
4,11,Night (10pm-5am),Weekday,291,13,82.91
5,11,Night (10pm-5am),Weekend,60,8,17.09
6,2,Night (10pm-5am),Weekday,430,10,82.53
7,2,Night (10pm-5am),Weekend,91,6,17.47
8,3,Night (10pm-5am),Weekday,3084,23,71.70
9,3,Night (10pm-5am),Weekend,1217,19,28.30


In [ ]:
import sqlite3 as db 
import os
import pandas as pd

# Connect to database using absolute path
db_path = '../sql_databases/processed_data.db'
conn = db.connect(db_path)


#Generate the percentage breakdown of incident type (e.g. crime, fire)

query = """
SELECT
  type AS incident_type,
  COUNT(*) AS total,
  ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(), 2) AS percentage
FROM (
  SELECT 'crime' AS type
  FROM sfpd_incidents
  WHERE "timestamp" IS NOT NULL

  UNION ALL

  SELECT 'fire' AS type
  FROM fire_incidents
  WHERE "Incident Date" IS NOT NULL
) t
GROUP BY type
ORDER BY total DESC;
"""



df = pd.read_sql_query(query, conn)
conn.close()

print(f"Total rows: {len(df)}")
print(df)

# Save results to a new database
results_db_path = '/Users/yejikwon/Desktop/MCIT/CIS5500/5500/final project/sql_databases/results.db'

# Create connection to results database
results_conn = db.connect(results_db_path)

# Save DataFrame to the database as a table
df.to_sql('incidents_type_percentage', results_conn, if_exists='replace', index=False)

results_conn.close()

print(f"\n✓ Results saved to {results_db_path}")

Total rows: 2
  incident_type   total  percentage
0          fire  148786        93.7
1         crime   10000         6.3

✓ Results saved to /Users/yejikwon/Desktop/MCIT/CIS5500/5500/final project/sql_databases/results.db


In [ ]:
import sqlite3 as db 
import os
import pandas as pd

# Connect to database using absolute path
db_path = '../sql_databases/processed_data.db'
conn = db.connect(db_path)


#Aggregate incidents by month across multiple years (crime and fire)
query = """
WITH crime AS (
  SELECT 
  	  date(strftime('%Y-%m-01', "timestamp")) AS month,
  	  COUNT(DISTINCT unique_key) AS cnt
 	 FROM sfpd_incidents
  WHERE "timestamp" IS NOT NULL
  GROUP BY 1
),
fire AS (
  SELECT 
    date(strftime('%Y-%m-01', "Incident Date")) AS month,
    COUNT(*) AS cnt
  FROM fire_incidents
  WHERE "Incident Date" IS NOT NULL
  GROUP BY 1
),
months AS(
    SELECT month FROM crime
    UNION
    SELECT month FROM fire)
SELECT
  m.month,
  COALESCE(c.cnt, 0) AS crime_cnt,
  COALESCE(f.cnt, 0) AS fire_cnt,
  COALESCE(c.cnt,0) + COALESCE(f.cnt,0) AS total_incidents
FROM months m
LEFT JOIN crime c ON c.month = m.month
LEFT JOIN fire f ON f.month = m.month
ORDER BY m.month;
"""

df = pd.read_sql_query(query, conn)
conn.close()

print(f"Total rows: {len(df)}")
print(df)

# Save results to a new database
results_db_path = '/Users/yejikwon/Desktop/MCIT/CIS5500/5500/final project/sql_databases/results.db'

# Create connection to results database
results_conn = db.connect(results_db_path)

# Save DataFrame to the database as a table
df.to_sql('incidents_by_month', results_conn, if_exists='replace', index=False)

results_conn.close()

print(f"\n✓ Results saved to {results_db_path}")

Total rows: 213
          month  crime_cnt  fire_cnt  total_incidents
0    2003-01-01         62         0               62
1    2003-02-01         47         0               47
2    2003-03-01         59         0               59
3    2003-04-01         72         0               72
4    2003-05-01         54         0               54
..          ...        ...       ...              ...
208  2020-05-01          0      2356             2356
209  2020-06-01          0      2224             2224
210  2020-07-01          0      2196             2196
211  2020-08-01          0      2051             2051
212  2020-09-01          0       483              483

[213 rows x 4 columns]

✓ Results saved to /Users/yejikwon/Desktop/MCIT/CIS5500/5500/final project/sql_databases/results.db


In [ ]:
import sqlite3 as db 
import os
import pandas as pd

# Connect to database using absolute path
db_path = '../sql_databases/processed_data.db'
conn = db.connect(db_path)


#Generate a list of the top 10 most frequently reported by crime categories by counting incidents in each category.

query = """
SELECT category, COUNT(*) AS cnt
FROM sfpd_incidents
GROUP BY category
ORDER BY cnt DESC
LIMIT 10;
"""


df = pd.read_sql_query(query, conn)
conn.close()

print("Top 10 crime categories:")
print(df)

# Save results to a new database
results_db_path = '/Users/yejikwon/Desktop/MCIT/CIS5500/5500/final project/sql_databases/results.db'

# Create connection to results database
results_conn = db.connect(results_db_path)

# Save DataFrame to the database as a table
df.to_sql('incidents_by_month', results_conn, if_exists='replace', index=False)

results_conn.close()

print(f"\n✓ Results saved to {results_db_path}")


Top 10 crime categories:
         category   cnt
0   LARCENY/THEFT  1786
1    NON-CRIMINAL  1240
2  OTHER OFFENSES  1215
3   VEHICLE THEFT   954
4         ASSAULT   786
5        BURGLARY   677
6   DRUG/NARCOTIC   546
7       VANDALISM   544
8  MISSING PERSON   456
9        WARRANTS   385

✓ Results saved to /Users/yejikwon/Desktop/MCIT/CIS5500/5500/final project/sql_databases/results.db
